# Pathway analysis

Pathway analysis methods provide meaningful insights to better understand how differentially expressed genes (DEG) impact biological processes.
Let's see how we can run **Over-Representation Analysis (ORA)** and **Gene Set Enrichment Analysis (GSEA)** on our data, using gseapy package. As the returned objects are pretty big, we delete them at the end of each section for the notebook to save memory.

For more details about the functions used here, you can refer to [gseapy documentation](https://gseapy.readthedocs.io), and for more information about GSEA in general (input data etc) : [gsea-msigdb website](https://www.gsea-msigdb.org/gsea/doc/GSEAUserGuideFrame.html)

In [ ]:
from pylluminator.utils import load_object
from pylluminator.dm import combine_p_values_stouffer
from pylluminator.utils import set_logger
from pylluminator.visualizations import visualize_gene

import numpy as np
import gseapy as gp
import networkx as nx
import matplotlib.pyplot as plt

set_logger('WARNING')  # set the verbosity level, can be DEBUG, INFO, WARNING, ERROR

## Data preparation

To run ORA and Pre-rank methods, we need to load DMPs or DMRs. If you haven't done it yet, check out notebook `3 - DMPs and DMRs`. 

For GSEA method, we use samples data directly - we use the samples stored in your DM object.

In [ ]:
my_dms = load_object('dms')  # load a DM object

In [ ]:
# get the genes associated with each probe
annotation_colname = 'genes'
gene_info = my_dms.samples.annotation.probe_infos[['probe_id', annotation_colname]].drop_duplicates().dropna()
# if some probes are associated to several genes, make it one row per gene. Make sure genes are upper case for GSEApy compatibilty
gene_info[annotation_colname] = gene_info[annotation_colname].apply(lambda x: x.upper().split(';'))
gene_info = gene_info.explode(annotation_colname).drop_duplicates()
gene_info.head()

For each gene, we use the DMPs or the DMRs to compute the fold change (average beta difference between the two conditions) and the significance (by combining the p-values of all associated probes using Stouffer's method).

In [ ]:
# chose whether to use DMPs or DMRs
input_type = 'DMP' 

# if we work on DMRs, add the probe_ids to the DMRs - for DMPs, just use the DMPs dataframe directly
dm_df = my_dms.dmr.join(my_dms.segments.reset_index().set_index('segment_id')) if input_type == 'DMR' else my_dms.dmp 
# add the gene information 
dm_df = dm_df.merge(gene_info, on='probe_id')

In [ ]:
# set the columns to use to select or rank genes (you can check available columns with dm_df.columns)
significance_colname = 'sample_type[T.PREC]_p_value_adjusted'
fold_change_colname = 'sample_type[T.PREC]_avg_beta_delta'
# set the column of the sample sheet that define the type of sample
class_colname = 'sample_type'

# keep only useful columns and remove NAs
dm_df = dm_df[[annotation_colname, fold_change_colname, significance_colname]].dropna()
# aggregate values for each gene
gene_fc_sig_df = dm_df.groupby(annotation_colname).agg({fold_change_colname: 'mean', significance_colname: combine_p_values_stouffer})

gene_fc_sig_df.head()

In [ ]:
dm_df[dm_df[significance_colname] < 0.05].sort_values(fold_change_colname, ascending=False)

In [ ]:
gene_fc_sig_df[gene_fc_sig_df[significance_colname] < 0.05].sort_values(fold_change_colname, ascending=False)

For GSEA, we compute the mean beta values per gene, for each sample

In [ ]:
probes_betas_df = my_dms.samples.get_betas().reset_index().set_index('probe_id').drop(columns=['type', 'channel', 'probe_type'])
genes_betas_df = probes_betas_df.join(gene_info.set_index('probe_id'), how='right').groupby(annotation_colname).mean()

# get the list of sample types, ordered like the beta dataframe
sample_info = my_dms.samples.sample_sheet.set_index(my_dms.samples.sample_label_name)
sample_types = sample_info.loc[probes_betas_df.columns, class_colname]

genes_betas_df.head()

In [ ]:
del dm_df
del probes_betas_df
del gene_info

### Gene set selection

You will first need to select the gene set(s) to use in you pathway analysis. You can browse [enrichr website](https://maayanlab.cloud/Enrichr/#libraries) or use the `gp.get_library_name()` function to list available libraries

In [ ]:
gene_sets = ['GO_Biological_Process_2025', 'KEGG_2021_Human']

### Organism specification

Specify the organism you're working on: Human, Mouse, Yeast, Fly, Fish or Worm

In [ ]:
organism = 'human'

## Over-Representation Analysis

### ORA without background

The minimal input you need to run an ORA is a list of differentially expressed genes (DEG) from your dataset. Here we use the threshold of `p-value < 0.02` and `abs(fold-change) > 0.85`

In [ ]:
deg = list(set(gene_fc_sig_df[(gene_fc_sig_df[significance_colname] < 0.02) & (abs(gene_fc_sig_df[fold_change_colname]) > 0.85)].index.values))
print(f'Number of genes selected: {len(deg)}/{len(set(gene_fc_sig_df.index))}\n')

enr = gp.enrichr(gene_list=deg, gene_sets=gene_sets, organism=organism) 

# output most significant results 
enr.results.sort_values(['Adjusted P-value', 'P-value']).head()

Adjusted P-Values are too high to extract any significant pathway. One reason can be that we are working on a limited number of probes (in notebook 4, we selected only 10% of the probes to compute the DMP, to speed up the demo - you can try with all the probes and). We can also try a different approach, ORA with background, to focus our analysis on the genes we are actually studying.

In [ ]:
# Here is the code to visualize the results. Use the cutoff parameter to adjust the p-value threshold
# ax = gp.dotplot(enr.results, x='Gene_set', size=3, top_term=5, title='ORA without background', xticklabels_rot=30, show_ring=True, figsize=(5,5))
# # use smaller fonts
# for item in ax.get_xticklabels() + ax.get_yticklabels():
#     item.set_fontsize(8)
# ax.xaxis.label.set_size(10)
# ax.title.set_size(15)

### ORA with background

By default, selected genes are tested against all genes available in the gene sets. For better results, it can be relevant to narrow it down to a list of genes of interest. 

Let's use the genes that were detected in our experiment as background genes, and see how it changes the result.

In [ ]:
background_genes = list(set(gene_fc_sig_df.index))
print('Number of selected genes: ', len(deg))
print('Number of background genes: ', len(background_genes), '\n')

enr_bg = gp.enrichr(gene_list=deg, gene_sets=gene_sets, organism=organism, background=background_genes) 
# output top 5 results results 
enr_bg.results.sort_values('Adjusted P-value')[:5]

Visualize the top 5 terms for each gene set, with a significance cutoff set at 0.1

In [ ]:
ax = gp.dotplot(enr_bg.results, x='Gene_set', size=2, top_term=5, figsize=(5, 5), y_order=True, title = 'ORA with background', xticklabels_rot=30, cutoff=0.1)
# use smaller fonts
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(8)
ax.xaxis.label.set_size(10)
ax.title.set_size(15)

In [ ]:
del deg
del enr
del enr_bg

## Pre-rank GSEA

As we have already calculate DMPs an DMRs, we can use them to rank the genes and use the pre-rank method provided by gseapy. 

Here we use the formula `sign(FC) * - log10(significance)` to rank the genes, that bring the most upregulated genes at the beginning of the list, the most downregulated at the end, and the less differentialy expressed genes in the middle.

In [ ]:
def rank_formula(row):
    if row[significance_colname] == 0: 
        return np.sign(row[fold_change_colname]) * np.inf
    return np.sign(row[fold_change_colname]) * -np.log10(row[significance_colname])

rank_data = gene_fc_sig_df.apply(rank_formula, axis=1).sort_values(ascending=False)

In [ ]:
# we chose a low permutation number to speed up the demo
pre_res = gp.prerank(rnk=rank_data, gene_sets=gene_sets, verbose=True, permutation_num=500, max_size=300, threads=4)

Order the pathways by their False Discovery Rate (FDR), and show the ones that have a FDR lower than 0.05

In [ ]:
pre_res.res2d = pre_res.res2d.sort_values('FDR q-val').reset_index(drop=True)
pre_res.res2d[pre_res.res2d['FDR q-val'] < 0.05]

Now we can visualize the most significant term

In [ ]:
term = pre_res.res2d.Term[0]
fig = pre_res.plot(terms=term, figsize=(10, 5))
# use smaller fonts
for ax in fig.axes:
    ax.set_xlabel(ax.get_xlabel(), fontsize=10)
    ax.set_ylabel(ax.get_ylabel(), fontsize=10)
_ = fig.suptitle(term, fontsize=15) 

Or visualize the top 5 terms on the same plot:

In [ ]:
fig = pre_res.plot(terms=pre_res.res2d.Term[:5],figsize=(3,4))
# use smaller fonts
for ax in fig.axes:
    ax.set_xlabel(ax.get_xlabel(), fontsize=10)
    ax.set_ylabel(ax.get_ylabel(), fontsize=10)

In [ ]:
ax = gp.dotplot(pre_res.res2d, column='FDR q-val', title='Pre-ranked GSEA', cmap=plt.cm.viridis, size=3, figsize=(3, 4), show_ring=True)
# use smaller fonts
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(8)
ax.xaxis.label.set_size(10)
ax.title.set_size(15)

In [ ]:
term_idx = 0  # chose the first term
genes = pre_res.res2d.Lead_genes[term_idx].split(";")
ax = gp.heatmap(df = genes_betas_df.loc[genes], z_score=0, title=pre_res.res2d.Term[term_idx], figsize=(7,6), yticklabels=False)
# use smaller fonts and show all labels
genes.reverse()  #for labels to be in the right order
ax.title.set_size(15)
_ = ax.set_xticks(range(len(genes_betas_df.columns)), labels=genes_betas_df.columns, rotation=30, ha="center",  fontsize=8)
_ = ax.set_yticks(range(len(genes)), labels=genes, va="bottom", fontsize=8)

Using pylluminator's function, we can go back to checking the beta values of the probes associated to a given gene

In [ ]:
visualize_gene(my_dms.samples, 'PAX2', figsize=(20, 6))

Finally, compute the enrichment map to show the relations between the detected relevant pathways

In [ ]:
nodes, edges = gp.enrichment_map(pre_res.res2d, cutoff=0.1)  # chose a higher p-value cutoff to show more nodes, default is 0.05

# build graph
G = nx.from_pandas_edgelist(edges,
                            source='src_idx',
                            target='targ_idx',
                            edge_attr=['jaccard_coef', 'overlap_coef', 'overlap_genes'])

nodes = nodes.loc[G.nodes.keys()]  # remove nodes that are not connected by any edge

fig, ax = plt.subplots(figsize=(10, 6), layout='constrained')

# init node coordinates
pos=nx.layout.bfs_layout(G, start=nodes.index[0])

# add a colorbar
nodes_cmap = plt.cm.RdYlBu
max_NES = max(abs(min(nodes.NES)), max(nodes.NES))
sm = plt.cm.ScalarMappable(cmap=nodes_cmap, norm=plt.Normalize(vmin=-max_NES, vmax=max_NES))
sm.set_array([])  
cbar = plt.colorbar(sm, ax=ax, shrink=0.25)
cbar.set_label('NES (Normalized Enrichment Score)', rotation=270, labelpad=15)

# draw nodes
nx.draw_networkx_nodes(G, pos=pos,
                       cmap=nodes_cmap, node_color=nodes.NES, vmin=-max_NES, vmax=max_NES,
                       margins=0.3, node_size=list(nodes.Hits_ratio *1000))
# draw node labels - wrap labels so that they don't overlap
max_length = 35
wrapped_labels = {k: "\n".join([v[i:i+max_length] for i in range(0, len(v), max_length)]) for k, v in nodes.Term.to_dict().items()}
nx.draw_networkx_labels(G, pos=pos, font_size= 6, labels=wrapped_labels)

# draw edges
edge_weight = nx.get_edge_attributes(G, 'jaccard_coef').values()
nx.draw_networkx_edges(G, pos=pos, width=list(map(lambda x: x*10, edge_weight)))

plt.axis('off')
plt.show()


In [ ]:
del gene_fc_sig_df
del pre_res
del my_dms

## GSEA

Since we are working with two groups (healthy control cells and prostate cancer cells), we can directly use the GSEA function on our 'raw' data (here, the genes beta values) with the corresponding phenotype dataframe (sample_types), to find the enriched pathways. Uncomment the following code to run the analysis.

In [ ]:
# gs = gp.GSEA(data=genes_betas_df.dropna(), gene_sets=gene_sets, classes=sample_types, permutation_num=1000)  
# gs.pheno_neg = 'PREC'  # control samples
# gs.pheno_pos = 'LNCAP'  # samples of prostate cancer cells
# gs.run()

Visualize the top 5 identified pathways

In [ ]:
# _ = gs.plot(gs.res2d.Term[:5])

Visualize, for a given pathway and for each gene, the samples z-score.

In [ ]:
# term_idx = 0  # chose the first term
# genes = gs.res2d.Lead_genes[term_idx].split(";")
# ax = gp.heatmap(df = genes_betas_df.loc[genes], z_score=0, title=gs.res2d.Term[term_idx], figsize=(8,4))